In [2]:
import numpy as np
import pandas as pd

# refine subthreshold lensed injections set

In [12]:
data= dict(np.load('lensed_data/sub_sub_lensed.npz')) 

In [13]:
data.keys()

dict_keys(['dist', 't0', 'tensor_snr', 'idx_of_pkl', 'm1', 'm2', 'ra', 'dec', 'incl', 'pol', 'phi0', 'dl_un'])

In [14]:
ntot=data['m1'].shape[0]
data['event_tag'] = np.arange(ntot).astype(int)
data['img_tag'] = np.vstack([np.zeros(ntot),np.ones(ntot)]).T.astype(int)

In [15]:
np.savez('lensed_data/sub_sub_lensed_refined.npz',**data)

# refine subthreshold unlensed injections set

In [17]:
data = pd.read_csv("unlensed_data/unlensed_sub_Dominik_powerlaw2_inj_samples_withsnr.dat",delimiter='\t')

ntot=data['m1z'].shape[0]

data['phi0']=2*np.pi*np.random.rand(ntot)

In [75]:
from numpy.core.defchararray import add

data['event_tag'] = add('sub_',np.arange(ntot).astype(str)).astype('U256')
data_dict = {}
for key in data.columns:
    data_dict[key] = data[key].values
np.savez('unlensed_data/sub_unlensed_refined.npz',**data_dict)

In [74]:
data_dict['event_tag']

array(['sub_0', 'sub_1', 'sub_2', ..., 'sub_1552', 'sub_1553', 'sub_1554'],
      dtype=object)

# create unlensed super sub pairs dataframe

In [64]:
sup_unlensed_data = np.load('../data/injection_pars/training/analytical_psd_Dominik_powerlaw2_inj_samples_withsnr_refined.npz')

In [65]:
sub_unlensed_data = np.load('unlensed_data/sub_unlensed_refined.npz')

In [66]:
sup_unlensed_data['m1z'].shape

(1004,)

In [67]:
sub_unlensed_data['m1z']

ValueError: Object arrays cannot be loaded when allow_pickle=False